import libraries(I provide all libs that I need when make this tasks, if you need some external import them here)

In [107]:
import pyspark
import os

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col
from pyspark.sql.functions import max, avg, min
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql.functions import when

create local SparkSession

In [3]:
spark = SparkSession.builder.appName("LocalSparkSession").master("local[*]").getOrCreate()
print(f"Spark Version: {spark.version}")
print(spark)

Spark Version: 3.4.3


read csv with inferschema

In [12]:
df = spark.read.csv("archive/ds_salaries.csv", header=True, inferSchema=True)

read csv one more time with the same code and you will see that it almostly don't take time, because info already in SparkSession and it will not read nothing
from this file

In [11]:
df = spark.read.csv("archive/ds_salaries.csv", header=True, inferSchema=True)

write schema of scv on screen

In [13]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- work_year: integer (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)



create schema of this scv

In [4]:
schema = StructType([
    StructField("_c0", IntegerType(), nullable=True),
    StructField("work_year", IntegerType(), nullable=True),
    StructField("experience_level", StringType(), nullable=True),
    StructField("employment_type", StringType(), nullable=True),
    StructField("job_title", StringType(), nullable=True),
    StructField("salary", IntegerType(), nullable=True),
    StructField("salary_currency", StringType(), nullable=True),
    StructField("salary_in_usd", IntegerType(), nullable=True),
    StructField("employee_residence", StringType(), nullable=True),
    StructField("remote_ratio", IntegerType(), nullable=True),
    StructField("company_location", StringType(), nullable=True),
    StructField("company_size", StringType(), nullable=True)
])

restart kernel without cleaning output and after restarting you need to initialize SparkSession, after initialize start execute only cells from cell with schema=
=StructType.... 
To restart kernel click Kernel, Restart.

read ds_salaries with predefined schema and compare results from this cell and cell with inferSchema

In [5]:
df = spark.read.csv("archive/ds_salaries.csv", header=True, schema=schema)

this happens because read operation is lazy(transformation), but if you use inferschema it start to be action that will create Spark Job, because Spark need to loop throw all file to check datatypes for all columns and this can harm to your code(if we compare to parquet, it will also go to check data types, but parquet provide meta information, so Spark will not go throw all file, he will just read meta information, but csv don't provide such meta information). Also header make Spark to create one more Spark Job to check first line
to define name of columns and remember to skeep it when reading. Actual reading start when you will use first action. More about Spark Jobs you will see in next topic

write schema of scv on screen one more time and compare with previous

In [6]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- work_year: integer (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)



now continue to work with one of the dataframes that you create

print data in dataframe using df.show

In [7]:
df.show()

+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
|_c0|work_year|experience_level|employment_type|           job_title|  salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
|  0|     2020|              MI|             FT|      Data Scientist|   70000|            EUR|        79833|                DE|           0|              DE|           L|
|  1|     2020|              SE|             FT|Machine Learning ...|  260000|            USD|       260000|                JP|           0|              JP|           S|
|  2|     2020|              SE|             FT|   Big Data Engineer|   85000|            GBP|       109024|                GB|          50|     

print data in dataframe using display(df.toPandas())

In [9]:
display(df.toPandas())

,_c0,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L
...,...,...,...,...,...,...,...,...,...,...,...,...
602,602,2022,SE,FT,Data Engineer,154000,USD,154000,US,100,US,M
603,603,2022,SE,FT,Data Engineer,126000,USD,126000,US,100,US,M
604,604,2022,SE,FT,Data Analyst,129000,USD,129000,US,0,US,M
605,605,2022,SE,FT,Data Analyst,150000,USD,150000,US,100,US,M


create df_job_title that consists from all job_titles without duplicates

In [17]:
df_job_title = df.select(df["job_title"]).dropDuplicates(["job_title"])

print all rows from df_job_titles without truncating jobs

In [18]:
df_job_title.show(truncate=False)

+----------------------------------------+
|job_title                               |
+----------------------------------------+
|3D Computer Vision Researcher           |
|Lead Data Engineer                      |
|Head of Machine Learning                |
|Data Specialist                         |
|Data Analytics Lead                     |
|Machine Learning Scientist              |
|Lead Data Analyst                       |
|Data Engineering Manager                |
|Staff Data Scientist                    |
|ETL Developer                           |
|Director of Data Engineering            |
|Product Data Analyst                    |
|Principal Data Scientist                |
|AI Scientist                            |
|Director of Data Science                |
|Machine Learning Engineer               |
|Lead Data Scientist                     |
|Machine Learning Infrastructure Engineer|
|Data Science Engineer                   |
|Machine Learning Manager                |
+----------

create  df_analytic that will consists from max, avg, min USD salaries for all job_titles using groupBy. name of fields is avg_salary, min_salary, max_salary

In [14]:
df_analytic = df.groupBy("job_title").agg(
    max("salary_in_usd").alias("max_salary"), 
    avg("salary_in_usd").alias("avg_salary"), 
    min("salary_in_usd").alias("min_salary")
    )

print all rows from df_analytic without trancating jobs

In [15]:
df_analytic.show(truncate=False)

+----------------------------------------+----------+------------------+----------+
|job_title                               |max_salary|avg_salary        |min_salary|
+----------------------------------------+----------+------------------+----------+
|3D Computer Vision Researcher           |5409      |5409.0            |5409      |
|Lead Data Engineer                      |276000    |139724.5          |56000     |
|Head of Machine Learning                |79039     |79039.0           |79039     |
|Data Specialist                         |165000    |165000.0          |165000    |
|Data Analytics Lead                     |405000    |405000.0          |405000    |
|Machine Learning Scientist              |260000    |158412.5          |12000     |
|Lead Data Analyst                       |170000    |92203.0           |19609     |
|Data Engineering Manager                |174000    |123227.2          |59303     |
|Staff Data Scientist                    |105000    |105000.0          |1050

now you need to add in df_analytic column row_id, that will show order of all job_titles depending on avg salary. they should be descending

In [20]:
window_spec = Window.orderBy(col("avg_salary").desc())
df_analytic = df_analytic.withColumn("row_id", row_number().over(window_spec))

print all data from df_analytic

In [22]:
df_analytic.show(truncate=False)

+----------------------------------+----------+------------------+----------+------+
|job_title                         |max_salary|avg_salary        |min_salary|row_id|
+----------------------------------+----------+------------------+----------+------+
|Data Analytics Lead               |405000    |405000.0          |405000    |1     |
|Principal Data Engineer           |600000    |328333.3333333333 |185000    |2     |
|Financial Data Analyst            |450000    |275000.0          |100000    |3     |
|Principal Data Scientist          |416000    |215242.42857142858|148261    |4     |
|Director of Data Science          |325000    |195074.0          |130026    |5     |
|Data Architect                    |266400    |177873.9090909091 |90700     |6     |
|Applied Data Scientist            |380000    |175655.0          |54238     |7     |
|Analytics Engineer                |205300    |175000.0          |135000    |8     |
|Data Specialist                   |165000    |165000.0          

it isn't beautifull, so we need to put now row_id on first place in df_analytic

In [23]:
df_analytic = df_analytic.select("row_id", "job_title", "max_salary", "avg_salary", "min_salary")

print df_analytic now

In [24]:
df_analytic.show(truncate=False)

+------+----------------------------------+----------+------------------+----------+
|row_id|job_title                         |max_salary|avg_salary        |min_salary|
+------+----------------------------------+----------+------------------+----------+
|1     |Data Analytics Lead               |405000    |405000.0          |405000    |
|2     |Principal Data Engineer           |600000    |328333.3333333333 |185000    |
|3     |Financial Data Analyst            |450000    |275000.0          |100000    |
|4     |Principal Data Scientist          |416000    |215242.42857142858|148261    |
|5     |Director of Data Science          |325000    |195074.0          |130026    |
|6     |Data Architect                    |266400    |177873.9090909091 |90700     |
|7     |Applied Data Scientist            |380000    |175655.0          |54238     |
|8     |Analytics Engineer                |205300    |175000.0          |135000    |
|9     |Data Specialist                   |165000    |165000.0   

here you need to create df_exp_lvl with the biggest usd_salary(biggest_salary) for each experience_level(you need to save all fields like in entire dataframe)

In [30]:
window_spec = Window.partitionBy("experience_level")
df_max_salary = df.withColumn("biggest_salary", max("salary_in_usd").over(window_spec))
df_exp_lvl = df_max_salary.filter(col("salary_in_usd") == col("biggest_salary"))

print here df_exp_lvl

In [58]:
df_exp_lvl.show(truncate=False)

+---+---------+----------------+---------------+-------------------------+------+---------------+-------------+------------------+------------+----------------+------------+--------------+
|_c0|work_year|experience_level|employment_type|job_title                |salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|biggest_salary|
+---+---------+----------------+---------------+-------------------------+------+---------------+-------------+------------------+------------+----------------+------------+--------------+
|37 |2020     |EN              |FT             |Machine Learning Engineer|250000|USD            |250000       |US                |50          |US              |L           |250000        |
|252|2021     |EX              |FT             |Principal Data Engineer  |600000|USD            |600000       |US                |100         |US              |L           |600000        |
|33 |2020     |MI              |FT             |Researc

create df_best that consists from rows where salary of guy same as biggest salary for other people in his exp_lvl and choose only columns: id, experience_level, biggest_salary, employee_residence

In [75]:
df_max_salary = df.groupBy("experience_level") \
                   .agg(max("salary_in_usd").alias("biggest_salary"))

df_best = df.join(df_max_salary, on="experience_level", how="inner") \
            .filter(col("salary_in_usd") == col("biggest_salary")) \
            .select(col("_c0").alias("id"), 
                    col("experience_level"), 
                    col("biggest_salary"), 
                    col("employee_residence"))


print df_best

In [76]:
df_best.show()

+---+----------------+--------------+------------------+
| id|experience_level|biggest_salary|employee_residence|
+---+----------------+--------------+------------------+
| 33|              MI|        450000|                US|
| 37|              EN|        250000|                US|
| 63|              SE|        412000|                US|
| 97|              MI|        450000|                US|
|252|              EX|        600000|                US|
+---+----------------+--------------+------------------+



drop duplicates if exist by experience_level

In [77]:
df_best = df_best.dropDuplicates()

print df_best

In [78]:
df_best.show()

+---+----------------+--------------+------------------+
| id|experience_level|biggest_salary|employee_residence|
+---+----------------+--------------+------------------+
| 63|              SE|        412000|                US|
| 33|              MI|        450000|                US|
| 97|              MI|        450000|                US|
|252|              EX|        600000|                US|
| 37|              EN|        250000|                US|
+---+----------------+--------------+------------------+



create df_new_best from df_best without id, and make the next: when exp_level = MI we want middle, when SE we want senior, else Null

In [81]:
df_new_best = df_best.select("experience_level", "biggest_salary", "employee_residence").withColumn("experience_level", 
                                 when(df_best.experience_level == "MI", "middle")
                                 .when(df_best.experience_level == "SE", "senior")
                                 .otherwise("Null"))

print df_new_best

In [82]:
df_new_best.show()

+----------------+--------------+------------------+
|experience_level|biggest_salary|employee_residence|
+----------------+--------------+------------------+
|          senior|        412000|                US|
|          middle|        450000|                US|
|          middle|        450000|                US|
|            Null|        600000|                US|
|            Null|        250000|                US|
+----------------+--------------+------------------+



write df_new_best like 1.csv and load then it to df_final

In [123]:

current_directory = os.getcwd()
print(current_directory)
file_name = os.path.join(current_directory, "1.csv")
df_new_best.write \
    .option("header", "true") \
    .option("delimiter", ";") \
    .mode("overwrite") \
    .csv(file_name)

d:\InnowiseGroup\Task_4_Spark\Spark\PySpark_Basics


Py4JJavaError: An error occurred while calling o1197.csv.
: java.lang.UnsatisfiedLinkError: org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Ljava/lang/String;I)Z
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:793)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1249)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1454)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.fs.ChecksumFileSystem.listStatus(ChecksumFileSystem.java:761)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter.getAllCommittedTaskPaths(FileOutputCommit

print df_final

filter df_final to delete experience_level where it Null, then join this table by biggest_salary(salary_in_usd) and employee_residence with entire df

print df_final

last task is to save in variable and then print this variable of the biggest salary_in_usd from df_final

It is the end of PySpark basics. In other lessons you will learn optimizations technics and how to make distributed system